In [1]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")


In [2]:
from langchain_core.messages import AnyMessage, AIMessage
from typing_extensions import TypedDict, Annotated

def add(left, right):
    return left + right

class State(TypedDict):
    # messages: list[AnyMessage]
    messages: Annotated[list[AnyMessage], add] # tells langgraph to use add function to combine messages
    extra_field: int

def node(state:State):
    # messages = state["messages"]
    new_message = AIMessage("Yo!")
    return {"messages":[new_message], "extra_field": 10}

In [3]:
from langgraph.graph import StateGraph
from IPython.display import display, Image

# State is already defined in a previous cell, so we can use it directly
builder = StateGraph(State)     # langgraph knows that messages should be combined using the `add` function
builder.add_node(node)
builder.set_entry_point("node")
graph = builder.compile()

# Show the Mermaid syntax (text representation)
print("Graph structure (Mermaid syntax):")
print(graph.get_graph().draw_mermaid())

Graph structure (Mermaid syntax):
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	node(node)
	__end__([<p>__end__</p>]):::last
	__start__ --> node;
	node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [4]:
from langchain_core.messages import HumanMessage

result = graph.invoke({"messages": [HumanMessage("Hi")]})
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi
================================== Ai Message ==================================

Yo!


In [5]:
# Runtime Configuration Example
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, END

class ConfigSchema(TypedDict):
    my_runtime_value: str

# Use different name to avoid conflict with earlier State class
class ConfigurableState(TypedDict):
    my_state_value: str

# Use different name to avoid conflict with earlier node function
def configurable_node(state: ConfigurableState, config: RunnableConfig):
    if config["configurable"]["my_runtime_value"] == "foo":
        return {"my_state_value": "bar"}
    elif config["configurable"]["my_runtime_value"] == "baz":
        return {"my_state_value": "qux"}
    else:
        raise ValueError("Invalid runtime value")

# Use different variable names to avoid conflicts
configurable_builder = StateGraph(ConfigurableState, config_schema=ConfigSchema)
configurable_builder.add_node("configurable_node", configurable_node)
configurable_builder.add_edge(START, "configurable_node")
configurable_builder.add_edge("configurable_node", END)

configurable_graph = configurable_builder.compile()

In [6]:
# Test the configurable graph with different runtime values

print("=== Testing with runtime value 'foo' ===")
result_foo = configurable_graph.invoke(
    {"my_state_value": "initial"}, 
    {"configurable": {"my_runtime_value": "foo"}}
)
print(f"State value: {result_foo['my_state_value']}")

print("\n=== Testing with runtime value 'baz' ===")
result_baz = configurable_graph.invoke(
    {"my_state_value": "initial"}, 
    {"configurable": {"my_runtime_value": "baz"}}
)
print(f"State value: {result_baz['my_state_value']}")

print("\n=== Testing with invalid value (will raise error) ===")
try:
    result_invalid = configurable_graph.invoke(
        {"my_state_value": "initial"}, 
        {"configurable": {"my_runtime_value": "invalid"}}
    )
except ValueError as e:
    print(f"Error caught: {e}")

=== Testing with runtime value 'foo' ===
State value: bar

=== Testing with runtime value 'baz' ===
State value: qux

=== Testing with invalid value (will raise error) ===
Error caught: Invalid runtime value
